In [21]:
import os
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import sys
import threading

In [22]:
tsp_dir = os.getcwd() + os.sep + 'TSPLIB' + os.sep

In [23]:
tsp_files_list = os.listdir(tsp_dir)

In [24]:
subprocess.run('make christofides', shell=True, capture_output=True)

CompletedProcess(args='make christofides', returncode=0, stdout=b"make: 'christofides' is up to date.\n", stderr=b'')

In [25]:
out = subprocess.run(['./christofides -f TSPLIB/gr17.tsp'], shell=True, capture_output=True)
output = out.stdout.decode('utf-8')
print(output)

OptimalSoln:	2085.000000
Simple:		1.053717	2197
Tri-Opt:	1.031655	2151
Tri-Comp:	1.031655	2151
Comp-Heur:	1.074820	2241



In [26]:
def parseOutputString(outputStr):
    lines = outputStr.split('\n')
    resultDict = {}
    for i in range(len(lines)-1):
        line = lines[i]
        s = line.split()
        if(i == 0):
            resultDict[i] = [s[1]]
        else:
            resultDict[i] = s[1:]
    return resultDict

In [27]:
parseOutputString(output)

{0: ['2085.000000'],
 1: ['1.053717', '2197'],
 2: ['1.031655', '2151'],
 3: ['1.031655', '2151'],
 4: ['1.074820', '2241']}

In [69]:
def getSupportedFilesList():
    tsp_files_list = os.listdir(tsp_dir)
    supportedFiles = []
    keyWord = 'EDGE_WEIGHT_SECTION'
    for filename in tsp_files_list:
        if(filename.find('.tsp') == -1):
            continue
        foundKeyWord = False
        file = open(tsp_dir + os.sep + filename, 'r')
        lines = file.readlines()
        for line in lines:
            if(line.find(keyWord) != -1):
                foundKeyWord = True
                break
        if(foundKeyWord == False):
            supportedFiles.append(filename)
    return supportedFiles

In [70]:
supported_tsp_files_list = getSupportedFilesList()

In [30]:
def launchChristofides(filename = 'gr48.tsp', perturbation_percentage = 0.0):
    filename = filename
    if(filename not in supported_tsp_files_list):
        print('WARNING: You are running the algorithm on an unsupported file, Perturbation does not work.')
    filename = 'TSPLIB' + os.sep + filename
    command = './christofides' + ' -f ' + filename + ' -p ' + str(perturbation_percentage)
#     print(command)
    out = subprocess.run([command], shell=True, capture_output=True)
    output = out.stdout.decode('utf-8')
    resultDict = parseOutputString(output)
#     print(resultDict)
    return resultDict

In [31]:
launchChristofides('gr431.tsp', 0.01)

{0: ['171414.000000'],
 1: ['1.125492', '192925'],
 2: ['1.070642', '183523'],
 3: ['1.050603', '180088'],
 4: ['1.079883', '185107']}

In [32]:
MAX_THREADS = 1

threads = []
for i in range(MAX_THREADS):
    t = threading.Thread(target = launchChristofides, args = ('gr431', 0.01))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

In [33]:
def launchChristofidesThreadSafe(resultDictList, listLock, filename = 'gr48', perturbation_percentage = 0.0):
    dictToAdd = launchChristofides(filename, perturbation_percentage)
    listLock.acquire()
    resultDictList.append(dictToAdd)
    listLock.release()
    return    

In [41]:
resultList = []
listLock = threading.Lock()
threads = []
for i in range(10):
    t = threading.Thread(target = launchChristofidesThreadSafe, args = (resultList, listLock, 'gr431.tsp', 0.01))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

print(len(resultList))
print(resultList)

10
[{0: ['171414.000000'], 1: ['1.126192', '193045'], 2: ['1.070228', '183452'], 3: ['1.050258', '180029'], 4: ['1.081569', '185396']}, {0: ['171414.000000'], 1: ['1.125392', '192908'], 2: ['1.070694', '183532'], 3: ['1.050918', '180142'], 4: ['1.086072', '186168']}, {0: ['171414.000000'], 1: ['1.124231', '192709'], 2: ['1.068267', '183116'], 3: ['1.050212', '180021'], 4: ['1.088808', '186637']}, {0: ['171414.000000'], 1: ['1.124762', '192800'], 2: ['1.070134', '183436'], 3: ['1.050748', '180113'], 4: ['1.093866', '187504']}, {0: ['171414.000000'], 1: ['1.118088', '191656'], 2: ['1.064522', '182474'], 3: ['1.051169', '180185'], 4: ['1.080040', '185134']}, {0: ['171414.000000'], 1: ['1.126098', '193029'], 2: ['1.069936', '183402'], 3: ['1.049943', '179975'], 4: ['1.082590', '185571']}, {0: ['171414.000000'], 1: ['1.118835', '191784'], 2: ['1.064785', '182519'], 3: ['1.051664', '180270'], 4: ['1.085827', '186126']}, {0: ['171414.000000'], 1: ['1.124850', '192815'], 2: ['1.068758', '18320

In [35]:
def runExpt(filename, perturbation_percentage, num_threads = 15):
    resultList = []
    listLock = threading.Lock()
    threads = []
    for i in range(num_threads):
        t = threading.Thread(target = launchChristofidesThreadSafe, args = (resultList, listLock, filename, perturbation_percentage))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    return resultList

In [36]:
ans = runExpt('gr431.tsp', 0.1, 1)
print(ans)

[{0: ['171414.000000'], 1: ['1.110779', '190403'], 2: ['1.062830', '182184'], 3: ['1.042750', '178742'], 4: ['1.113328', '190840']}]


In [96]:
def getAvgList(inpDict=[]):
    l = [0.0 for i in range(5)]
    n = len(inpDict)
    for d in inpDict:
        try:
            l[0] += float(d[0][0])
        except:
#             print(d)
            n -= 1
            print('failed', end='\t')
            continue
        for i in range(4):
            l[i+1] += float(d[i+1][1])
    for i in range(5):
        l[i] = l[i]/n
    return l

In [38]:
getAvgList(ans)

[171414.0, 190403.0, 182184.0, 178742.0, 190840.0]

In [39]:
NUM_OBSERVATIONS = 100
ppList = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]
data = {}
# for i in range(len(supported_tsp_files_list)):
for i in range(1):
    filename = supported_tsp_files_list[i]
    for j in range(len(ppList)):
#     for j in range(1):
        pp = ppList[j]
        output = runExpt(filename, pp, NUM_OBSERVATIONS)
#         print(output)
        l = getAvgList(output)
        data[(filename, pp)] = []
        for k in range(5):
            data[(filename, pp)].append(l[k])
print(data)

{('a280.tsp', 0.0): [2579.0, 2909.0, 2711.0, 2617.0, 2815.0], ('a280.tsp', 0.01): [2579.0, 2963.32, 2723.46, 2635.8, 2778.36], ('a280.tsp', 0.02): [2579.0, 2967.14, 2727.36, 2636.86, 2780.47], ('a280.tsp', 0.05): [2579.0, 2970.73, 2730.62, 2642.2, 2767.53], ('a280.tsp', 0.1): [2579.0, 2965.82, 2737.27, 2649.93, 2787.12], ('a280.tsp', 0.2): [2579.0, 2924.53, 2742.66, 2663.02, 2820.93], ('a280.tsp', 0.3): [2579.0, 2921.23, 2745.9, 2669.18, 2835.6]}


In [75]:
def getNumFromString(s):
    numStr = ''
    for c in list(s):
        if(c.isdigit()):
            numStr += (c)
    return int(numStr)

In [76]:
def getData(MAX_N = 1000, NUM_OBSERVATIONS = 15, NUM_FILES = len(supported_tsp_files_list), ppList = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]):
    data = {}
    for i in range(NUM_FILES):
        filename = supported_tsp_files_list[i]
        if(getNumFromString(filename) > MAX_N):
            continue
        print(filename, end='\t')
        for j in range(len(ppList)):
            pp = ppList[j]
            output = runExpt(filename, pp, NUM_OBSERVATIONS)
            print(pp, end = '\t')
            l = getAvgList(output)
            data[(filename, pp)] = []
            for k in range(5):
                data[(filename, pp)].append(l[k])
        print('done')
    return data

In [97]:
getData(500, 15, 100)

a280.tsp	0.0	0.01	0.02	0.05	0.1	0.2	0.3	done
att48.tsp	0.0	0.01	0.02	0.05	0.1	0.2	0.3	done
berlin52.tsp	0.0	0.01	0.02	0.05	0.1	0.2	0.3	done
bier127.tsp	0.0	0.01	0.02	0.05	0.1	0.2	0.3	done
burma14.tsp	0.0	0.01	0.02	0.05	0.1	0.2	0.3	done
ch130.tsp	0.0	0.01	0.02	0.05	0.1	0.2	0.3	done
ch150.tsp	0.0	0.01	0.02	0.05	0.1	0.2	0.3	done
d198.tsp	0.0	0.01	0.02	0.05	0.1	0.2	0.3	done
d493.tsp	0.0	failed	failed	failed	failed	failed	failed	failed	failed	failed	failed	failed	failed	failed	failed	failed	

ZeroDivisionError: float division by zero

In [98]:
a = runExpt('d493.tsp', 0.01, 15)
getAvgList(a)

failed	failed	failed	failed	failed	failed	failed	failed	failed	failed	failed	

[35002.0, 38767.0, 37262.75, 36233.5, 38477.25]